In [1]:
#Cargamos las librerias que vamos a utilizar para la limpieza de datos
import json
import ast
import numpy as np
import pandas as pd
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import nltk
import pyarrow

In [6]:
data = []  # Inicializar la lista para almacenar los datos

# Leer datos desde el archivo JSON y cargarlos en la lista 'data'
with open('C:\\Users\\Dr-Pc\\OneDrive\\Escritorio\\PI MLOps - STEAM\\output_steam_games.json', 'r') as file:
    for line in file:
        data.append(json.loads(line))

# Crear el DataFrame a partir de la lista 'data'
user_games_final = pd.DataFrame(data)

In [7]:
#aquí renombramos las columnas para que sea vean más prolijas y esteticas
user_games_final.rename(columns={'publisher': 'Publisher'}, inplace=True)
user_games_final.rename(columns={'genres': 'Genres'}, inplace=True)
user_games_final.rename(columns={'app_name': 'App Name'}, inplace=True)
user_games_final.rename(columns={'url': 'URL'}, inplace=True)
user_games_final.rename(columns={'tags': 'Tags'}, inplace=True)
user_games_final.rename(columns={'reviews_url': 'Reviews URL'}, inplace=True)
user_games_final.rename(columns={'specs': 'Specs'}, inplace=True)
user_games_final.rename(columns={'price': 'Price'}, inplace=True)
user_games_final.rename(columns={'early_access': 'Early Access'}, inplace=True)
user_games_final.rename(columns={'id': 'Item ID'}, inplace=True)
user_games_final.rename(columns={'developer': 'Developer'}, inplace=True)
user_games_final.rename(columns={'title': 'Item Name'}, inplace=True)

In [8]:
#creamos la columna "Release Date" que llenaremos con datos extraidos de la columna "release_date"
user_games_final['Release Date'] = user_games_final['release_date'].str.extract(r'(\d{4})')
#eliminamos la columna que ya no utilizamos
user_games_final.drop('release_date' , axis = 1, inplace = True)

In [9]:
#borramos datos nulos
user_games_final.dropna(subset=['Release Date'], inplace=True)

In [10]:
#borramos los duplicados 
user_games_final.drop_duplicates(subset="URL", inplace=True)

In [11]:
#borramos las filas en donde todas las columnas sean nulos
user_games_final.dropna(how="all",inplace=True)

In [12]:
# Verificar los tipos de datos de las columnas
print(user_games_final.dtypes)

# Convertir las columnas a tipo cadena y eliminar corchetes y comillas, conservando los valores nulos
columns_to_clean = ['Specs', 'Tags', 'Genres']

for column in columns_to_clean:
    user_games_final[column] = user_games_final[column].apply(lambda x: ', '.join([i.replace("[", "").replace("]", "").replace("'", "").replace("\"", "") for i in x]) if isinstance(x, list) else x)

Publisher       object
Genres          object
App Name        object
Item Name       object
URL             object
Tags            object
Reviews URL     object
Specs           object
Price           object
Early Access    object
Item ID         object
Developer       object
Release Date    object
dtype: object


In [13]:
# Capitalizar la primera letra de cada palabra en la columna 'Specs', pasando los valores nulos de largo
user_games_final['Specs'] = user_games_final['Specs'].apply(lambda x: [word.title() for word in x] if isinstance(x, list) else x if pd.notnull(x) else None)

# Capitalizar la primera letra de cada palabra en la columna 'Genres', pasando los valores nulos de largo
user_games_final['Genres'] = user_games_final['Genres'].apply(lambda x: [word.title() for word in x] if isinstance(x, list) else x if pd.notnull(x) else None)

# Capitalizar la primera letra de cada palabra en la columna 'Tags', pasando los valores nulos de largo
user_games_final['Tags'] = user_games_final['Tags'].apply(lambda x: [word.title() for word in x] if isinstance(x, list) else x if pd.notnull(x) else None)

In [14]:
# Convertir Early Access a tipo de datos booleano y manejar valores nulos
user_games_final['Early Access'] = user_games_final['Early Access'].apply(lambda x: True if x == 'True' else (False if x == 'False' else x) if x is not None else x)

# Convierte los valores no nulos a cadena de texto y mantiene los valores nulos
user_games_final['Reviews URL'] = user_games_final['Reviews URL'].apply(lambda x: str(x).strip() if pd.notnull(x) else None)

# Convierte los valores no nulos a cadena de texto y mantiene los valores nulos
user_games_final['URL'] = user_games_final['URL'].apply(lambda x: str(x).strip() if pd.notnull(x) else None)

In [35]:
#vemos como van quedando los datos y la cantidad 
user_games_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11264 entries, 1 to 22527
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Genres        11264 non-null  object
 1   Item Name     11264 non-null  object
 2   Item ID       11264 non-null  object
 3   Developer     11264 non-null  object
 4   Release Date  11264 non-null  object
dtypes: object(5)
memory usage: 528.0+ KB


In [17]:
#borramos los datos nulos de las siguientes columnas 
user_games_final.dropna(subset=['Publisher'], inplace=True)
user_games_final.dropna(subset=['Genres'], inplace=True)
user_games_final.dropna(subset=['App Name'], inplace=True)
user_games_final.dropna(subset=['URL'], inplace=True)
user_games_final.dropna(subset=['Tags'], inplace=True)
user_games_final.dropna(subset=['Reviews URL'], inplace=True)
user_games_final.dropna(subset=['Specs'], inplace=True)
user_games_final.dropna(subset=['Price'], inplace=True)
user_games_final.dropna(subset=['Item ID'], inplace=True)
user_games_final.dropna(subset=['Developer'], inplace=True)
user_games_final.dropna(subset=['Early Access'], inplace=True)

In [18]:
#reseteamos en index del archivo
user_games_final.reset_index(drop=True, inplace=True)

In [19]:
#borramos la columa URL
user_games_final.drop('URL' , axis = 1, inplace = True)

In [20]:
#borramos la columna Reviews URL
user_games_final.drop('Reviews URL' , axis = 1, inplace = True)

In [32]:
#borramos las siguientes columnas
user_games_final.drop('Publisher' , axis = 1, inplace = True)
user_games_final.drop('App Name' , axis = 1, inplace = True)
user_games_final.drop('URL' , axis = 1, inplace = True)
user_games_final.drop('Tags' , axis = 1, inplace = True)
user_games_final.drop('Reviews URL' , axis = 1, inplace = True)
user_games_final.drop('Specs' , axis = 1, inplace = True)
user_games_final.drop('Price' , axis = 1, inplace = True)
user_games_final.drop('Early Access' , axis = 1, inplace = True)

In [22]:
#renombramos app name a item name
user_games_final.rename(columns={'App Name': 'Item Name'}, inplace=True)

In [33]:
#vemos como van quedando los datos y las columnas
user_games_final.head()

,Genres,Item Name,Item ID,Developer,Release Date
0,"Action, Casual, Indie, Simulation, Strategy",Lost Summoner Kitty,761140,Kotoshiro,2018
1,"Free to Play, Indie, RPG, Strategy",Ironbound,643980,Secret Level SRL,2018
2,"Casual, Free to Play, Indie, Simulation, Sports",Real Pool 3D - Poolians,670290,Poolians.com,2017
3,"Action, Adventure, Casual",弹炸人2222,767400,彼岸领域,2017
4,"Action, Adventure, Simulation",Battle Royale Trainer,772540,Trickjump Games Ltd,2018


In [47]:
#creamos un archivo parquet del dataframe a parquet
user_games_final.to_parquet('user_games_final.parquet')